In [262]:
import sys

import pandas as pd

from pathlib import Path
from typing import Union, List
from from_root import from_root

sys.path.insert(0, str(from_root("src")))

from read_and_write_docs import read_excel_sheets, read_rds

In [263]:
models = ["gemma-3-270m", "gpt2", "Qwen2.5-0.5B-Instruct", "Qwen2.5-1.5B-Instruct"]

corpuses = ["Wiki", "Perverted Justice"]

data_types = ["training", "test"]

base_loc = "/Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs"

metadata_base_loc = "/Volumes/BCross/datasets/author_verification"

In [264]:
expected_by_corpus_dt = {
    ("Wiki", "training"): 5000,
    ("Wiki", "test"): 672,
    ("Perverted Justice", "training"): 3000,
    ("Perverted Justice", "test"): 574,
}

In [265]:
def list_xlsx_files(
    directory: Union[str, Path],
    *,
    recursive: bool = False,
    include_temp: bool = False,
    sort: bool = True,
) -> List[Path]:
    """
    Return all .xlsx files in a directory as a list of Paths.

    Parameters
    ----------
    directory : str | Path
        Directory to search.
    recursive : bool
        If True, search subdirectories too.
    include_temp : bool
        If True, include Excel temp files like "~$something.xlsx".
    sort : bool
        If True, sort results by path.

    Returns
    -------
    List[Path]
        Paths to .xlsx files.
    """
    directory = Path(directory)
    if not directory.exists():
        raise FileNotFoundError(f"Directory not found: {directory}")
    if not directory.is_dir():
        raise NotADirectoryError(f"Not a directory: {directory}")

    pattern = "**/*.xlsx" if recursive else "*.xlsx"
    files = list(directory.glob(pattern))

    if not include_temp:
        files = [p for p in files if not p.name.startswith("~$")]

    if sort:
        files = sorted(files)

    return files


In [266]:
def compare_counts_to_expected_map(base_loc, data_types, corpuses, models, expected_by_corpus_dt, *, recursive=False):
    """
    Compare actual .xlsx counts on disk vs expected counts (expected varies by (corpus, data_type)).
    Assumes directory layout: {base_loc}/{data_type}/{corpus}/{model}/raw
    """
    base_loc = Path(base_loc)
    rows = []

    for data_type in data_types:
        for corpus in corpuses:
            expected = expected_by_corpus_dt.get((corpus, data_type), None)

            for model in models:
                data_loc = base_loc / data_type / corpus / model / "raw"

                if data_loc.exists():
                    actual = len(list_xlsx_files(data_loc, recursive=recursive))
                else:
                    actual = 0

                delta = (actual - expected) if expected is not None else None

                rows.append({
                    "data_type": data_type,
                    "corpus": corpus,
                    "model": model,
                    "expected_num_files": expected,
                    "actual_num_files": actual,
                    "delta": delta,
                    "missing": (expected - actual) if expected is not None and actual < expected else 0 if expected is not None else None,
                    "extra": (actual - expected) if expected is not None and actual > expected else 0 if expected is not None else None,
                    "status": (
                        "NOT_STARTED" if expected is not None and actual == 0
                        else "COMPLETED" if expected is not None and actual == expected
                        else "MISSING" if expected is not None and actual < expected
                        else "EXTRA" if expected is not None and actual > expected
                        else "NO_EXPECTATION"
                    ),
                })

    df = (
        pd.DataFrame(rows)
        .sort_values(["data_type", "corpus", "model"])
        .reset_index(drop=True)
    )

    # Handy lookup: (data_type, corpus, model) -> row dict
    lookup = {
        (r.data_type, r.corpus, r.model): r._asdict()
        for r in df.itertuples(index=False)
    }

    return df, lookup

In [267]:
actual_df, actual_lookup = compare_counts_to_expected_map(base_loc, data_types, corpuses, models, expected_by_corpus_dt)

In [268]:
actual_df

,data_type,corpus,model,expected_num_files,actual_num_files,delta,missing,extra,status
0,test,Perverted Justice,Qwen2.5-0.5B-Instruct,574,574,0,0,0,COMPLETED
1,test,Perverted Justice,Qwen2.5-1.5B-Instruct,574,574,0,0,0,COMPLETED
2,test,Perverted Justice,gemma-3-270m,574,574,0,0,0,COMPLETED
3,test,Perverted Justice,gpt2,574,491,-83,83,0,MISSING
4,test,Wiki,Qwen2.5-0.5B-Instruct,672,672,0,0,0,COMPLETED
5,test,Wiki,Qwen2.5-1.5B-Instruct,672,672,0,0,0,COMPLETED
6,test,Wiki,gemma-3-270m,672,672,0,0,0,COMPLETED
7,test,Wiki,gpt2,672,672,0,0,0,COMPLETED
8,training,Perverted Justice,Qwen2.5-0.5B-Instruct,3000,0,-3000,3000,0,NOT_STARTED
9,training,Perverted Justice,Qwen2.5-1.5B-Instruct,3000,0,-3000,3000,0,NOT_STARTED


In [269]:
completed_data = actual_df[actual_df['status'] == 'COMPLETED']
completed_data

,data_type,corpus,model,expected_num_files,actual_num_files,delta,missing,extra,status
0,test,Perverted Justice,Qwen2.5-0.5B-Instruct,574,574,0,0,0,COMPLETED
1,test,Perverted Justice,Qwen2.5-1.5B-Instruct,574,574,0,0,0,COMPLETED
2,test,Perverted Justice,gemma-3-270m,574,574,0,0,0,COMPLETED
4,test,Wiki,Qwen2.5-0.5B-Instruct,672,672,0,0,0,COMPLETED
5,test,Wiki,Qwen2.5-1.5B-Instruct,672,672,0,0,0,COMPLETED
6,test,Wiki,gemma-3-270m,672,672,0,0,0,COMPLETED
7,test,Wiki,gpt2,672,672,0,0,0,COMPLETED


In [270]:
def compare_complete_to_metadata(metadata_base_loc, data_type, corpus, model, excel_files):
    
    metadata_loc = f"{metadata_base_loc}/{data_type}/doc_level_metadata.rds"
    
    metadata = read_rds(metadata_loc)
    metadata = metadata[metadata['corpus'] == corpus]
    metadata['scoring_model'] = model
    
    file_names = [ef.name for ef in excel_files]
    # df with filename + completed=True
    df = pd.DataFrame({
        "filename": file_names,
        "completed": True
    })

    # left join onto metadata_df and fill missing completed with False
    metadata_df = (
        metadata
        .merge(df, on="filename", how="left")
    )

    metadata_df["completed"] = metadata_df["completed"].fillna(False).astype(bool)
    metadata_df["scored"] = False
    
    return metadata_df

In [271]:
def create_problem_complete_metadata(metadata: pd.DataFrame) -> pd.DataFrame:
    """
    Groups by (data_type, corpus, scoring_model, problem) and returns:
      - num_files: total rows
      - files_completed: count where completed == True
      - files_scored: count where scored == True
      - problem_completed: True if num_files == files_scored
    """
    group_cols = ["data_type", "corpus", "scoring_model", "problem"]

    out = (
        metadata
        .groupby(group_cols, dropna=False)
        .agg(
            num_files=("filename", "size"),
            files_completed=("completed", lambda s: int(s.fillna(False).astype(bool).sum())),
            files_scored=("scored", lambda s: int(s.fillna(False).astype(bool).sum())),
        )
        .reset_index()
    )

    out["problem_completed"] = out["num_files"] == out["files_scored"]
    return out

In [276]:
def build_and_save_token_level_raw_scores(
    completed_df: pd.DataFrame,
    base_loc: str | Path,
    metadata_base_loc: str | Path,
    *,
    sheet_name: str = "metadata",
    output_name: str = "token_level_raw_scores.xlsx",
    recursive: bool = False,
    engine: str | None = None,
    overwrite: bool = False
) -> None:
    """
    For each row in completed_df, read all .xlsx files in:
        {base_loc}/{data_type}/{corpus}/{model}/raw
    Extract `sheet_name`, concat, sort by sample_id then min_token_size,
    and save to:
        {base_loc}/{data_type}/{corpus}/{model}/{output_name}

    Skips if output file already exists.
    """
    base_loc = Path(base_loc)

    required_cols = {"data_type", "corpus", "model"}
    missing = required_cols - set(completed_df.columns)
    if missing:
        raise ValueError(f"completed_df is missing required columns: {sorted(missing)}")

    for row in completed_df.itertuples(index=False):
        data_type = getattr(row, "data_type")
        corpus = getattr(row, "corpus")
        model = getattr(row, "model")

        raw_dir = base_loc / data_type / corpus / model / "raw"
        out_path = raw_dir.parent / output_name  # removes /raw
        metadata_out_path = base_loc / data_type / corpus / model / "raw_problem_metadata.xlsx"
        summary_metadata_out_path = base_loc / data_type / corpus / model / "raw_problem_completed_metadata.xlsx"
        
        # skip if output already exists
        if not overwrite:
            if out_path.exists():
                print(f"SKIP (exists): {out_path}")
                continue

        # if raw dir missing / empty, skip
        if not raw_dir.exists():
            print(f"SKIP (no dir): {raw_dir}")
            continue

        excel_files = list_xlsx_files(raw_dir, recursive=recursive)
        if not excel_files:
            print(f"SKIP (no files): {raw_dir}")
            continue

        base_metadata = compare_complete_to_metadata(metadata_base_loc, data_type, corpus, model, excel_files)
        
        combined_metadata = []

        for ef in excel_files:
            f_name = ef.name
            try:
                data = read_excel_sheets(ef, [sheet_name])
                combined_metadata.append(data[sheet_name])
                
                # ✅ mark as scored if read succeeded
                base_metadata.loc[base_metadata["filename"] == f_name, "scored"] = True
            except Exception as e:
                print(f"  WARN: failed reading {sheet_name} from {ef}: {e}")

        if not combined_metadata:
            print(f"SKIP (no readable sheets): {raw_dir}")
            continue

        results = (
            pd.concat(combined_metadata, ignore_index=True)
            .sort_values(["sample_id", "min_token_size"], ascending=[True, True], kind="mergesort")
            .reset_index(drop=True)
        )

        # insert data_type before corpus, scoring_model after corpus
        if "corpus" in results.columns:
            corpus_idx = results.columns.get_loc("corpus")

            if "data_type" in results.columns:
                results.drop(columns=["data_type"], inplace=True)
            results.insert(corpus_idx, "data_type", data_type)

            corpus_idx = results.columns.get_loc("corpus")  # re-fetch
            if "scoring_model" in results.columns:
                results.drop(columns=["scoring_model"], inplace=True)
            results.insert(corpus_idx + 1, "scoring_model", model)
            
        # move problem before known_author (always move; adjust index if problem was before)
        if "problem" in results.columns and "known_author" in results.columns:
            problem_idx = results.columns.get_loc("problem")
            known_author_idx = results.columns.get_loc("known_author")

            problem_col = results.pop("problem")

            # if problem was before known_author, known_author shifted left by 1 after pop
            if problem_idx < known_author_idx:
                known_author_idx -= 1

            results.insert(known_author_idx, "problem", problem_col)
        
        # ensure parent exists, then save
        out_path.parent.mkdir(parents=True, exist_ok=True)
        results.to_excel(out_path, index=False)
        print(f"SAVED: {out_path}  (rows={len(results)})")
        
        # Also want to save the metadata
        base_metadata.to_excel(metadata_out_path, index=False)
        
        summary_metadata = create_problem_complete_metadata(base_metadata)
        summary_metadata.to_excel(summary_metadata_out_path, index=False)

In [277]:
build_and_save_token_level_raw_scores(
    completed_data,
    base_loc,
    metadata_base_loc,
    sheet_name = "metadata",
    output_name = "token_level_raw_scores.xlsx",
    overwrite=True
)

SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Perverted Justice/Qwen2.5-0.5B-Instruct/token_level_raw_scores.xlsx  (rows=1721)
SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Perverted Justice/Qwen2.5-1.5B-Instruct/token_level_raw_scores.xlsx  (rows=1721)
SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Perverted Justice/gemma-3-270m/token_level_raw_scores.xlsx  (rows=1864)


<positron-console-cell-277>:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Wiki/Qwen2.5-0.5B-Instruct/token_level_raw_scores.xlsx  (rows=1606)


<positron-console-cell-277>:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Wiki/Qwen2.5-1.5B-Instruct/token_level_raw_scores.xlsx  (rows=1606)


<positron-console-cell-277>:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Wiki/gemma-3-270m/token_level_raw_scores.xlsx  (rows=2086)


<positron-console-cell-277>:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Wiki/gpt2/token_level_raw_scores.xlsx  (rows=1973)


In [274]:
manual_df = actual_df[
    (actual_df['data_type'] == 'test')
    & (actual_df['corpus'] == 'Perverted Justice')
    & (actual_df['model'] == 'gpt2')
]

manual_df

,data_type,corpus,model,expected_num_files,actual_num_files,delta,missing,extra,status
3,test,Perverted Justice,gpt2,574,491,-83,83,0,MISSING


In [275]:
build_and_save_token_level_raw_scores(
    manual_df,
    base_loc,
    metadata_base_loc,
    sheet_name = "metadata",
    output_name = "token_level_raw_scores.xlsx"
)

<positron-console-cell-275>:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


SAVED: /Volumes/BCross/av_datasets_experiments/ngram_masking_logrpobs/test/Perverted Justice/gpt2/token_level_raw_scores.xlsx  (rows=1513)
